In [1]:
import pandas as pd
import numpy as np
import sys


In [2]:


# Load your data (replace 'your_data.csv' with your actual file)
time_period = "14-00--14-10"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')

print(data_stream.groupby(["sector"]).size())

date_column = 'ts_event'

time_delta = 0
batch_results = []
data_stream[date_column] = pd.to_datetime(data_stream[date_column])
previous_event_timestamp = data_stream[date_column].iloc[0]
previous_check_timestamp = data_stream[date_column].iloc[0]




time_start = pd.Timestamp('2024-10-15 14:00:10.00', tz='UTC')
time_end = pd.Timestamp('2024-10-15 14:00:11.00', tz='UTC')


data_stream[date_column] = pd.to_datetime(data_stream[date_column])
# get data between time_start and time_end
data_stream = data_stream[(data_stream[date_column] >= time_start) & (data_stream[date_column] <= time_end)]




sector
Basic Materials              3598
Communication Services     413290
Consumer Cyclical          589540
Consumer Defensive         199084
Energy                     148997
Financial Services         119199
Healthcare                 175482
Industrials                  5771
Technology                1210127
dtype: int64


In [3]:

print(len(data_stream))

# check ts_event column
print(data_stream[date_column].head())
print(data_stream[date_column].tail())
print(data_stream[date_column].min())
print(data_stream[date_column].max())


10244
74010   2024-10-15 14:00:10.000005390+00:00
74011   2024-10-15 14:00:10.000014516+00:00
74012   2024-10-15 14:00:10.000037823+00:00
74013   2024-10-15 14:00:10.000050011+00:00
74014   2024-10-15 14:00:10.000050508+00:00
Name: ts_event, dtype: datetime64[ns, UTC]
84249   2024-10-15 14:00:10.999595611+00:00
84250   2024-10-15 14:00:10.999595611+00:00
84251   2024-10-15 14:00:10.999595892+00:00
84252   2024-10-15 14:00:10.999595892+00:00
84253   2024-10-15 14:00:10.999628058+00:00
Name: ts_event, dtype: datetime64[ns, UTC]
2024-10-15 14:00:10.000005390+00:00
2024-10-15 14:00:10.999628058+00:00


In [4]:
print(data_stream.groupby("sector").size())

sector
Basic Materials            108
Communication Services    2443
Consumer Cyclical         1925
Consumer Defensive         618
Energy                     492
Financial Services         248
Healthcare                 655
Industrials                 36
Technology                3719
dtype: int64


In [11]:
import pandas as pd
import numpy as np


time_period = "14-00--14-10"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
pre_data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')

time_start = pd.Timestamp('2024-10-15 14:00:10.30', tz='UTC')
time_end = pd.Timestamp('2024-10-15 14:00:11.00', tz='UTC')


pre_data_stream[date_column] = pd.to_datetime(pre_data_stream[date_column])
# get data between time_start and time_end
pre_data_stream = pre_data_stream[ (pre_data_stream[date_column] <= time_start)]

print("len predatastream: ", len(pre_data_stream))
print(pre_data_stream.groupby("sector").size())



time_period = "14-00--14-10"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')



data_stream[date_column] = pd.to_datetime(data_stream[date_column])
# get data between time_start and time_end
data_stream = data_stream[(data_stream[date_column] >= time_start) & (data_stream[date_column] <= time_end)]

print("len of data stream start to end:", len(data_stream))

date_column = 'ts_event'


len predatastream:  76601
sector
Basic Materials             289
Communication Services    11694
Consumer Cyclical          8868
Consumer Defensive         4889
Energy                     3514
Financial Services         2944
Healthcare                 4461
Industrials                 132
Technology                39810
dtype: int64
len of data stream start to end: 7653


In [12]:
import secrets
#
# # Sort by timestamp (if not already sorted)
# data_stream = data_stream.sort_values(date_column).reset_index(drop=True)
remove_stock_fraction = 0.1

# Function to downsample technology stocks
def downsample_tech_stocks(row):
    if row['ts_event'] >= time_start and row['sector'] == 'Communication Services':
        if secrets.randbelow(100) / 100 > remove_stock_fraction:  # Randomly exclude some tech stocks
            return None  # Exclude this row
    return row
# Apply the downsampling
modified_data = data_stream.apply(downsample_tech_stocks, axis=1).dropna()

print("len of modified_data:", len(modified_data))


result_data =  pd.concat([pre_data_stream, modified_data])
print("len of result_data: ", len(result_data))
# Verify the result
print(result_data.groupby("sector").size())


len of modified_data: 6071
len of result_data:  82672
sector
Basic Materials             289
Communication Services    11694
Consumer Cyclical          8868
Consumer Defensive         4889
Energy                     3514
Financial Services         2944
Healthcare                 4461
Industrials                 132
Technology                39810
dtype: int64


In [13]:
result_data[date_column].min()

Timestamp('2024-10-15 14:00:00.000258375+0000', tz='UTC')

In [14]:
result_data[date_column].max()

Timestamp('2024-10-15 14:00:10.299495769+0000', tz='UTC')

In [15]:
time_start1 = pd.Timestamp('2024-10-15 14:00:08.00', tz='UTC')

result_data = result_data[(result_data[date_column] >= time_start1)]
print("len of result data: ", len(result_data))
print(result_data.groupby("sector").size())


len of result data:  12519
sector
Basic Materials             11
Communication Services    2395
Consumer Cyclical         1683
Consumer Defensive         603
Energy                     781
Financial Services         448
Healthcare                 902
Industrials                106
Technology                5590
dtype: int64


In [16]:
result_data.to_csv(f"random_filtered_data_fraction_{int(remove_stock_fraction*10)}.csv", index=False)